In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import logging
import numpy as np

import xarray as xr
import xarray.ufuncs as xu
import xrft
import pandas as pd

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

from cross_defPlot import *
from cross_compute import *

from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, silence_logs='error', protocol = 'tcp://',
                      host = '137.129.155.67')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:8787/status,
Dashboard: http://137.129.155.67:8787/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: tcp://137.129.155.67:44303,Workers: 1
Dashboard: http://137.129.155.67:8787/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://137.129.155.67:37423,Total threads: 4
Dashboard: http://137.129.155.67:47183/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
### Parameter for the analysis
year = np.arange(2001,2019)
latitude = 10.5
longitude = 179

wave_TCWV = ['TCWV_TD', 'TCWV_MJO', 'TCWV_Kelvin', 'TCWV_Rossby', 'TCWV_MRG', 'TCWV_WIG', 'TCWV_EIG']  
wave_OLR = ['OLR_TD', 'OLR_MJO', 'OLR_Kelvin', 'OLR_Rossby', 'OLR_MRG', 'OLR_WIG', 'OLR_EIG']
wave = np.concatenate((wave_TCWV, wave_OLR), axis =0 )
coeff = 1

In [3]:
indir_data_RAW = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_ANOMALY/TCWV/'
indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/TCWV/'


ds_RAW = xr.open_mfdataset(indir_data_RAW+'*.nc', chunks = {'time':1}, parallel=True)
ds_RAW = ds_RAW.rename({'latitude':'lat', 'longitude':'lon'})
ds_RAW = ds_RAW.sel(lon = 0.5, method = 'nearest')

ds_FILTER = xr.open_mfdataset(indir_data_FILTERED + '*.nc', chunks = {'time':1}, parallel=True)
ds_FILTER = ds_FILTER.sel(lon = 0.5, method = 'nearest')


ds_RAW_TCWV = ds_RAW.reindex(lat=list(reversed(ds_RAW.lat)))
ds_FILTER_TCWV = ds_FILTER.reindex(lat=list(reversed(ds_FILTER.lat)))
ds_FILTER_TCWV

<xarray.Dataset>
Dimensions:      (lat: 321, time: 84736)
Coordinates:
  * lat          (lat) float64 -40.0 -39.75 -39.5 -39.25 ... 39.5 39.75 40.0
  * time         (time) datetime64[ns] 1991-01-01 ... 2019-12-31T21:00:00
    lon          float32 0.5
Data variables:
    TCWV_Kelvin  (time, lat) float64 dask.array<chunksize=(1, 321), meta=np.ndarray>
    TCWV_Rossby  (time, lat) float64 dask.array<chunksize=(1, 321), meta=np.ndarray>
    TCWV_MJO     (time, lat) float64 dask.array<chunksize=(1, 321), meta=np.ndarray>
    TCWV_MRG     (time, lat) float64 dask.array<chunksize=(1, 321), meta=np.ndarray>
    TCWV_EIG     (time, lat) float64 dask.array<chunksize=(1, 321), meta=np.ndarray>
    TCWV_WIG     (time, lat) float64 dask.array<chunksize=(1, 321), meta=np.ndarray>
    TCWV_TD      (time, lat) float64 dask.array<chunksize=(1, 321), meta=np.ndarray>

In [4]:
indir_data_RAW = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_ANOMALY/OLR/'
indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/OLR/'


ds_RAW = xr.open_mfdataset(indir_data_RAW+'anom_OLR_brut_ERA5*.nc', chunks = {'time':1}, parallel=True)
ds_RAW_OLR = ds_RAW.sel(lon = 0, method = 'nearest')

ds_FILTER = xr.open_mfdataset(indir_data_FILTERED + '*.nc', chunks = {'time':1}, parallel=True)
ds_FILTER_OLR = ds_FILTER.sel(lon = 0, method = 'nearest')
ds_FILTER_OLR

<xarray.Dataset>
Dimensions:     (time: 52592, lat: 60)
Coordinates:
  * time        (time) datetime64[ns] 2001-01-01 ... 2018-12-31T21:00:00
  * lat         (lat) float32 -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
    lon         float32 0.5
Data variables:
    OLR_Kelvin  (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_Rossby  (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_MJO     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_MRG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_EIG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_WIG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_TD      (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>

In [11]:
_ds_FILTER_TCWV = ds_FILTER_TCWV.sel(time = ds_FILTER_OLR.time)
_ds_FILTER_TCWV = _ds_FILTER_TCWV.sel(lat = ds_FILTER_OLR.lat)
_ds_FILTER_TCWV

<xarray.Dataset>
Dimensions:      (lat: 60, time: 52592)
Coordinates:
  * lat          (lat) float64 -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
  * time         (time) datetime64[ns] 2001-01-01 ... 2018-12-31T21:00:00
    lon          float32 0.5
Data variables:
    TCWV_Kelvin  (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_Rossby  (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_MJO     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_MRG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_EIG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_WIG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_TD      (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>

In [12]:
ds_FILTER = xr.merge([_ds_FILTER_TCWV, ds_FILTER_OLR])
ds_RAW = xr.merge([ds_RAW_TCWV, ds_RAW_OLR])
# ds_FILTER = ds_FILTER.chunks({'time' : 1})
ds_FILTER

<xarray.Dataset>
Dimensions:      (lat: 60, time: 52592)
Coordinates:
  * lat          (lat) float64 -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
  * time         (time) datetime64[ns] 2001-01-01 ... 2018-12-31T21:00:00
    lon          float32 0.5
Data variables: (12/14)
    TCWV_Kelvin  (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_Rossby  (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_MJO     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_MRG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_EIG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    TCWV_WIG     (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    ...           ...
    OLR_Rossby   (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_MJO      (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_MRG      (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_EIG      (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_WIG      (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>
    OLR_TD       (time, lat) float64 dask.array<chunksize=(1, 60), meta=np.ndarray>

In [ ]:
ds_std_mean = ds_FILTER.std(dim = 'time').compute()

In [ ]:
wave_TCWV = ['TCWV_TD', 'TCWV_MJO', 'TCWV_Kelvin', 'TCWV_Rossby', 'TCWV_MRG', 'TCWV_EIG', 'TCWV_WIG']  
wave_OLR = ['OLR_TD', 'OLR_MJO', 'OLR_Kelvin', 'OLR_Rossby', 'OLR_MRG', 'OLR_EIG','OLR_WIG']


wave = np.concatenate((wave_TCWV, wave_OLR), axis =0 )
coeff = 1
### Compute standard deviation
for w in wave:
    ds_FILTER = addNegative(ds_FILTER, ds_std_mean, w)
    
ds_FILTER = ds_FILTER.compute()
ds_FILTER

In [ ]:
ds_FILTER.to_netcdf('/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/ANALYSIS/LON0/OLR_TCWV_wave.nc')